TO DO:
X 1) Tokenize the input into list of words 
X 2) Lemmatize the list of words 
X 3) Perform POS tagging
X 4) Parsed sentence = Parse using regular expressions
X 5) If table.attribute ∈ Parsed sentence
     X  a) Extract them
     X  b) Call SQLmap()

In [1]:
import pandas as pd
import os  

print(os.listdir("./input"))

['test.csv']


In [2]:
test = pd.read_csv('./input/test.csv')

print('Shape of test set: ', test.shape)
test.head(5)

Shape of test set:  (4, 1)


,text
0,fare price of class2 Shatabdi express from Mum...
1,What are the available trains that travel from...
2,What is fare for class3 Dadar passenger from D...
3,Available trains from Chennai to Mumbai runnin...


In [3]:
import re
import string
import nltk

lemmatizedarr = []
POStaggedarr = []
modifiedSentence = []

# preparing list of stop words
stop_words = set(nltk.corpus.stopwords.words('english'))
exclude_words = set(("from", "to"))
new_stop_words = stop_words.difference(exclude_words)

from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

from nltk.tokenize import word_tokenize

row_count = test.shape[0]

# Text Preprocessing
for row in range(0, row_count):

    # extract sentence from csv file
    sentence = test.iloc[row]['text']
    print("\nNatural Language input is as follows: ")
    print(sentence, "\n")

    # remove punctuation
    print("Remove Punctuations- ")
    sentence = sentence.translate(sentence.maketrans("","",string.punctuation))
    print(sentence, "\n")
    modifiedSentence.append(sentence)

    # tokenize the sentence
    print("Tokenize The Sentence-")
    sentence = word_tokenize(sentence)
    print(sentence, "\n")

    # removing stop words
    print("Removing Stop Words-")
    words_in_sentence = [word for word in sentence if not word in new_stop_words]
    print(words_in_sentence, "\n")

    # lemmatizing the tokens
    print("Lemmatizing The Tokens-")
    for index,word in enumerate(words_in_sentence):
        words_in_sentence[index] = lemmatizer.lemmatize(word)
    print(words_in_sentence, "\n")
    lemmatizedarr.append(words_in_sentence)

    # POS tagging of lemmatized tokens
    tagged = nltk.pos_tag(words_in_sentence)
    print("POS Tagging Of Lemmatized Tokens- ")
    print(tagged, "\n")
    POStaggedarr.append(tagged)

    print("--------------------------------------------------------------------------------------------------------")



Natural Language input is as follows: 
fare price of class2 Shatabdi express from Mumbai to Goa on 2nd march 2021. 

Remove Punctuations- 
fare price of class2 Shatabdi express from Mumbai to Goa on 2nd march 2021 

Tokenize The Sentence-
['fare', 'price', 'of', 'class2', 'Shatabdi', 'express', 'from', 'Mumbai', 'to', 'Goa', 'on', '2nd', 'march', '2021'] 

Removing Stop Words-
['fare', 'price', 'class2', 'Shatabdi', 'express', 'from', 'Mumbai', 'to', 'Goa', '2nd', 'march', '2021'] 

Lemmatizing The Tokens-
['fare', 'price', 'class2', 'Shatabdi', 'express', 'from', 'Mumbai', 'to', 'Goa', '2nd', 'march', '2021'] 

POS Tagging Of Lemmatized Tokens- 
[('fare', 'JJ'), ('price', 'NN'), ('class2', 'NN'), ('Shatabdi', 'NNP'), ('express', 'NN'), ('from', 'IN'), ('Mumbai', 'NNP'), ('to', 'TO'), ('Goa', 'VB'), ('2nd', 'CD'), ('march', 'NN'), ('2021', 'CD')] 

--------------------------------------------------------------------------------------------------------

Natural Language input is as fol

In [4]:
# Regular expressions for extraction of attributes from the sentence


# Extract class of fare done
def Fare_Identifier(sentence):
    fare = re.search('fare' , sentence)
    if fare:
        d = re.search('class[123]', sentence)
        return d[0]
    return None


# Extract train name done
def Train_Name_Identifier(sentence):
    d = re.search('([^ \r\n]+) express | ([^ \r\n]+) mail | ([^ \r\n]+) passenger', sentence)
    if d:
        return d[0]
    else: 
        return None


# Extract source and destination Using Pos tagged list 
def Source_Destination_Identifier(posarr):
    source = ""
    destination = ""
    nextflagsource = False
    nextflagdestination = False

    for i in posarr:
        if nextflagsource:
            source = i[0]
            nextflagsource = False
        if nextflagdestination:
            destination = i[0]
            nextflagdestination = False

        if i[1] == "IN":
            nextflagsource = True
        if i[1] == "TO":
            nextflagdestination = True

    return source, destination


# Extract date
def Date_Identifier(sentence):
    day = None
    month = None
    year = None

    # extracting word before and the word of month 
    day_month = re.search(r"([^ \r\n]+) (jan+)| ([^ \r\n]+) (feb+)| ([^ \r\n]+) (mar+)| ([^ \r\n]+)(apr+)| ([^ \r\n]+)(may+)| ([^ \r\n]+)(jun+)| ([^ \r\n]+)(jul+)| ([^ \r\n]+)(aug+)| ([^ \r\n]+)(sep+)| ([^ \r\n]+)(oct+)| ([^ \r\n]+)(nov+)| ([^ \r\n]+)(dec+)", sentence)
    
    if day_month:
        #extracting day from day and month
        day = re.search(r'[0123456789]',day_month[0])[0]
        month = re.search(r"(jan+)|(feb+)|(mar+)|(apr+)|(may+)|(jun+)|(jul+)|(aug+)|(sep+)|(oct+)|(nov+)|(dec+)", day_month[0])[0]
   
    # extracting word after and the word month
    d = re.search(r"(january) (\w+)| (february) (\w+)| (march) (\w+)| (april) (\w+)| (may) (\w+)| (june) (\w+)| (july) (\w+)| (august) (\w+)| (september) (\w+)| (october) (\w+)| (november) (\w+)| (december) (\w+)", sentence, re.I)
    
    if d:
        #extracting year from month and year
        year = re.search(r'[0123456789]+',d[0])[0]

    m_np = 0
    if month == "jan":
        m_np = 1
    elif month == "feb":
        m_np = 2
    elif month == "mar":
        m_np = 3
    elif month == "apr":
        m_np = 4
    elif month == "may":
        m_np = 5
    elif month == "jun":
        m_np = 6
    elif month == "jul":
        m_np = 7
    elif month == "aug":
        m_np = 8
    elif month == "sep":
        m_np = 9
    elif month == "oct":
        m_np = 10
    elif month == "nov":
        m_np = 11
    else:
        m_np = 12

    return day + "/" + str(m_np) + "/" + year


Date_Identifier(modifiedSentence[2])

'5/1/2021'

In [5]:

# function for query formation from extracted attributes
def sql_map(value, source, destination, date, fare):
    query= "SELECT "
    if fare:
        query = query + "fare_" + fare + " "
    else:
        query = query + "train_no, " + "train_name "
    query = query + "FROM railways.train WHERE "
    if value:
        query = query + "train_name = " + value
    if source and destination:
        query = query + " source_stn = " + source + " destination_stn = " + destination
    if date:
        query = query + " date = " + date
    
    print(query)


# driver code 
for row in range(0, row_count):
    value = Train_Name_Identifier(modifiedSentence[row])
    source, destination = Source_Destination_Identifier(POStaggedarr[row])
    date = Date_Identifier(modifiedSentence[row])
    fare = Fare_Identifier(modifiedSentence[row])
    print("Query " + str(row) + ": \n")
    sql_map(value, source, destination, date, fare)    
    print('\n\n')



Query 0: 

SELECT fare_class2 FROM railways.train WHERE train_name = Shatabdi express  source_stn = Mumbai destination_stn = Goa date = 2/3/2021



Query 1: 

SELECT train_no, train_name FROM railways.train WHERE  source_stn = Delhi destination_stn = Mumbai date = 1/2/2021



Query 2: 

SELECT fare_class3 FROM railways.train WHERE train_name =  Dadar passenger source_stn = Dadar destination_stn = Chiplun date = 5/1/2021



Query 3: 

SELECT train_no, train_name FROM railways.train WHERE  source_stn = Chennai destination_stn = Mumbai date = 2/2/2020



